In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

# 데이터를 pandas dataframe으로 불러온다
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# print(train.head())

# 크게 관련 없는 columns를 삭제한다
# Ticket, Fare, Cabin, PasengerId
train.drop(["Ticket", "Fare", "Cabin", "PassengerId", "Name"], axis=1, inplace=True)
test.drop(["Ticket", "Fare", "Cabin", "PassengerId", "Name"], axis=1, inplace=True)


In [2]:
# NaN값이 들어있는 columns를 확인한다
train.isnull().sum(axis=0)

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

* Age값에 Nan값이 꽤 많이 들어 있는 것으로 보인다
* 일단은 Train Data의 평균을 이용해서 NaN 값을 채우는 것이 좋아보인다
* 데이터의 통일성을 위해 Test Data 또한 Train Data의 Mean 값을 이용해 보간한다.

In [3]:
train["Age"].fillna(train["Age"].mean(), inplace=True)
test["Age"].fillna(train["Age"].mean(), inplace=True)


In [4]:
# 각 column별로 값을 확인한다
for index in train.columns:
    print(train[index].value_counts())
    print()

0    549
1    342
Name: Survived, dtype: int64

3    491
1    216
2    184
Name: Pclass, dtype: int64

male      577
female    314
Name: Sex, dtype: int64

29.699118    177
24.000000     30
22.000000     27
18.000000     26
28.000000     25
            ... 
55.500000      1
53.000000      1
20.500000      1
23.500000      1
0.420000       1
Name: Age, Length: 89, dtype: int64

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64

0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

S    644
C    168
Q     77
Name: Embarked, dtype: int64



In [5]:
# 넘파이배열로 변환
train = train.values
test = test.values
set = [train,test]

In [6]:
# male, female 값을 각각 0과 1로 인코딩한다
sex_i = 3
embark_i = 7
for data in set:
    sex_i -= 1
    embark_i -= 1
    for i in range(data.shape[0]):
        print(data[i,sex_i])
        if data[i,sex_i] == 'male' :
            data[i,sex_i] = 0
        else :
            data[i,sex_i] = 1
        if data[i,embark_i] == 'S':
            data[i,embark_i] = 0
        elif data[i,embark_i] == 'Q':
            data[i,embark_i] = 1
        else :
            data[i,embark_i] = 2

male
female
female
female
male
male
male
male
female
female
female
female
male
male
female
female
male
male
female
female
male
male
female
male
female
female
male
male
female
male
male
female
female
male
male
male
male
male
female
female
female
female
male
female
female
male
male
female
male
female
male
male
female
female
male
male
female
male
female
male
male
female
male
male
male
male
female
male
female
male
male
female
male
male
male
male
male
male
male
female
male
male
female
male
female
female
male
male
female
male
male
male
male
male
male
male
male
male
female
male
female
male
male
male
male
male
female
male
male
female
male
female
male
female
female
male
male
male
male
female
male
male
male
female
male
male
male
male
female
male
male
male
female
female
male
male
female
male
male
male
female
female
female
male
male
male
male
female
male
male
male
female
male
male
male
male
female
male
male
male
male
female
male
male
male
male
female
female
male
male
male
male
female
male
male
mal

In [9]:
# 데이터 형변환
train = train.astype(np.float64)
test = test.astype(np.float64)

한 sample은 총 6개의 feature를 가지고 있다.

3개의 Fully Connected Layer를 이용한 신경망을 이용해보자


In [11]:
# 훈련용 변수와 정답 레이블을 분리한다
x_train = train[:,1:]
y_train = train[:,0]

In [13]:
# keras모델 설계
model = models.Sequential()
model.add(layers.Dense(16, activation="relu", input_shape=(6,)))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(
    x_train, y_train, epochs=200, batch_size=100, validation_split=0.2, shuffle = True
)

history_dict = history.history
history_dict.keys()

Epoch 1/20
609/712 [========================>.....] - ETA: 1s - loss: 0.8966 - accuracy: 0.6174

KeyboardInterrupt: 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# ‘bo’는 파란색 점을 의미합니다
plt.plot(epochs, loss, 'bo', label='Training loss')
# ‘b’는 파란색 실선을 의미합니다
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # 그래프를 초기화합니다
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()